In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 데이터 로드 및 전처리 함수
train_df = pd.read_csv('C:\Workspace/power_consumption_comp\data/train.csv')
test_df = pd.read_csv('C:\Workspace/power_consumption_comp\data/test.csv')

#결측값을 0으로 채웁니다 안바꿔도 됨
train_df = train_df.fillna(0)

#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train_df['month'] = train_df['일시'].apply(lambda x : int(x[4:6]))
train_df['day'] = train_df['일시'].apply(lambda x : int(x[6:8]))
train_df['time'] = train_df['일시'].apply(lambda x : int(x[9:11]))

train_x = train_df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
train_y = train_df['전력소비량(kWh)']

test_df['month'] = test_df['일시'].apply(lambda x : int(x[4:6]))
test_df['day'] = test_df['일시'].apply(lambda x : int(x[6:8]))
test_df['time'] = test_df['일시'].apply(lambda x : int(x[9:11]))

test_x = test_df.drop(columns=['num_date_time', '일시'])

# 데이터셋 클래스 정의
class MyDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        if y is not None:
            self.y = torch.tensor(y.values, dtype=torch.float32)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if hasattr(self, 'y'):
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

# DataLoader 객체 생성
batch_size = 8
train_dataset = MyDataset(train_x, train_y)
test_dataset = MyDataset(test_x)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

# Attention 모델 생성    
# class AttentionModel(nn.Module):
#     def __init__(self, input_size, hidden_size):
#         super(AttentionModel, self).__init__()
#         self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
#         self.fc = nn.Linear(hidden_size, 1)
#         self.attention = nn.Sequential(
#             nn.Linear(hidden_size, 1),
#             nn.Softmax(dim=1)
#         )

#     def forward(self, x):
#         rnn_out, _ = self.rnn(x)
#         attention_weights = self.attention(rnn_out)
#         attention_out = (attention_weights * rnn_out).sum(dim=1)
#         output = self.fc(attention_out)
#         return output

# Attention 모델 생성    
class AttentionModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(AttentionModel, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.attention = nn.Sequential(
            nn.Linear(hidden_size, 1),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        rnn_out, _ = self.rnn(x)
        attention_weights = self.attention(rnn_out)
        attention_out = (attention_weights * rnn_out).sum(dim=1)
        output = self.fc(attention_out)
        return output



# 학습 및 테스트 데이터를 기반으로 모델 학습
#model = AttentionModel(input_size=len(train_x.columns), hidden_size=64)
# Attention 모델 생성
model = AttentionModel(input_size=len(train_x.columns), hidden_size=8, output_size=8)
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    model.train()
    for inputs, targets in train_dataloader:
        optimizer.zero_grad()
        predictions = model(inputs)
        #loss = loss_fn(predictions, targets.unsqueeze(1))
        loss = loss_fn(predictions, targets)
        loss.backward()
        optimizer.step()

    # 모델 테스트
    model.eval()
    with torch.no_grad():
        test_loss = 0.0
        for inputs in test_dataloader:
            predictions = model(inputs)
            # 여기서 예측값 활용하여 필요한 작업 수행
            # 예를 들어, 예측값을 파일에 저장하거나 시각화하는 등의 작업을 수행할 수 있습니다.
            test_loss += loss_fn(predictions, targets).item()

    print(f"Epoch {epoch+1}/{epochs}, Test Loss: {test_loss / len(test_dataloader):.4f}")


Epoch 1/10, Test Loss: 2309921.0000
Epoch 2/10, Test Loss: 3240815.7500
Epoch 3/10, Test Loss: 1573836.5008
Epoch 4/10, Test Loss: 1062816.3750
Epoch 5/10, Test Loss: 852154.8125
Epoch 6/10, Test Loss: 3397189.2500
Epoch 7/10, Test Loss: 2242123.0000
Epoch 8/10, Test Loss: 3051053.7506
Epoch 9/10, Test Loss: 2560385.2500
Epoch 10/10, Test Loss: 1547432.1694
